[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/ai-rag-lab/)


# Step 1: Setup prerequisites

In [ ]:
import os
import sys
from pymongo import MongoClient

# Add parent directory to path to import from utils
sys.path.append(os.path.join(os.path.dirname(os.getcwd())))
from utils import track_progress, set_env

In [ ]:
# If you are using your own MongoDB Atlas cluster, use the connection string for your cluster here
MONGODB_URI = os.environ.get("MONGODB_URI")
# Initialize a MongoDB Python client
mongodb_client = MongoClient(MONGODB_URI)
# Check the connection to the server
mongodb_client.admin.command("ping")

In [ ]:
# Track progress of key steps-- DO NOT CHANGE
track_progress("cluster_creation", "ai_rag_lab")

In [ ]:
# Set the URL for our AI model proxy
PROXY_ENDPOINT = os.environ.get("PROXY_ENDPOINT")

In [ ]:
# Set the passkey provided by your workshop instructor
PASSKEY = "replace-with-passkey"

In [ ]:
# Obtain a Voyage API key from our AI model proxy and set it as an environment variable-- DO NOT CHANGE
set_env(["voyageai"], PASSKEY)

# Step 2: Load the dataset

In [ ]:
import json

In [ ]:
with open("../data/mongodb_docs.json", "r") as data_file:
    json_data = data_file.read()

docs = json.loads(json_data)

In [ ]:
# Note the number of documents in the dataset
len(docs)

In [ ]:
# Preview a document to understand its structure
docs[0]

# Step 3: Chunk and embed the data


In [ ]:
# You might see a warning after running this cell-- You can ignore it
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Dict, List
import voyageai
from tqdm import tqdm

In [ ]:
# Common list of separators for text data
separators = ["\n\n", "\n", " ", "", "#", "##", "###"]

In [ ]:
# Use the `RecursiveCharacterTextSplitter` from LangChain to first split a piece of text on the list of `separators` above.
# Then recursively merge them into tokens until the specified chunk size is reached.
# For text data, you typically want to keep 1-2 paragraphs (~200 tokens) in a single chunk.
# Set chunk overlap to 0 for contextualized embeddings, otherwise 15-20% of chunk size.
# The `model_name` parameter indicates which encoder to use for tokenization, in this case GPT-4's encoder.
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4", separators=separators, chunk_size=200, chunk_overlap=0
)

📚 https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html

In [ ]:
def get_chunks(doc: Dict, text_field: str) -> List[Dict]:
    """
    Chunk up a document.

    Args:
        doc (Dict): Parent document to generate chunks from.
        text_field (str): Text field to chunk.

    Returns:
        List[Dict]: List of chunked documents.
    """
    # Extract the field to chunk from `doc`
    text = doc[text_field]
    # Split `text` using the `split_text` method of the `text_splitter` object above
    chunks = <CODE_BLOCK_1>
    return chunks

In [ ]:
# Initialize the Voyage AI client
vo = voyageai.Client()

📚 https://docs.voyageai.com/docs/contextualized-chunk-embeddings#approach-2-contextualized-chunk-embeddings

In [ ]:
def get_embeddings(content: List[str], input_type: str) -> List[float] | List[List[float]]:
    """
    Get contextualized embeddings for each chunk.

    Args:
        content (List[str]): List of chunked texts or the user query as a list
        input_type (str): Type of input, either "document" or "query" 

    Returns:
        List[float] | List[List[float]]: Contextualized embeddings
    """
    # Use the `contextualized_embed` method of the Voyage AI API to get contextualized embeddings for each chunk with the following arguments:
    # inputs: `content` wrapped in another list
    # model: `voyage-context-3`
    # input_type: `input_type`
    embds_obj = <CODE_BLOCK_2>
     # If `input_type` is "document", there is a single result with multiple embeddings, one for each chunk
    if input_type == "document":
        embeddings = [emb for r in embds_obj.results for emb in r.embeddings]
    # If `input_type` is "query", there is a single result with a single embedding
    if input_type == "query":
        embeddings = embds_obj.results[0].embeddings[0]
    return embeddings

In [ ]:
embedded_docs = []
# Iterate through `docs` from Step 2
for doc in tqdm(docs):
    # Use the `get_chunks` function to chunk up the "body" field in each document
    chunks = <CODE_BLOCK_3>
    # Pass all the `chunks` to the `get_embeddings` function to get contextualized embeddings for each chunk
    # `input_type` should be set to "document" since we are embedding the "documents" for RAG
    chunk_embeddings = <CODE_BLOCK_4>
    # For each chunk, create a new document with the original metadata
    # Replace the `body` with the chunk content and add an `embedding` field
    for chunk, embedding in zip(chunks, chunk_embeddings):
        # Create a new document by copying the original document
        chunk_doc = doc.copy()
        # Replace the `body` field of `chunk_doc` with the chunk content
        <CODE_BLOCK_5>
        # Add an `embedding` field to `chunk_doc`, containing the embedding for this chunk
        <CODE_BLOCK_6>
        # Append `chunk_doc` to `embedded_docs`
        embedded_docs.append(chunk_doc)

In [ ]:
# Notice that the length of `embedded_docs` is greater than the length of `docs` from Step 2 above
# This is because each document in `docs` has been split into multiple chunks
len(embedded_docs)

In [ ]:
# Preview a chunked document to understand its structure
# Note that the structure looks similar to the original docs, except the `body` field now contains smaller chunks of text
# Each document also has an additional `embedding` field
embedded_docs[0]

# Step 4: Ingest data into MongoDB


### **Do not change the values assigned to the variables below**

In [ ]:
# Name of the database -- Change if needed or leave as is
DB_NAME = "mongodb_genai_devday_rag"
# Name of the collection -- Change if needed or leave as is
COLLECTION_NAME = "knowledge_base"
# Name of the vector search index -- Change if needed or leave as is
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"

In [ ]:
# Connect to the `COLLECTION_NAME` collection.
collection = mongodb_client[DB_NAME][COLLECTION_NAME]

In [ ]:
# Bulk delete all existing records from the collection defined above
collection.delete_many({})

📚 https://pymongo.readthedocs.io/en/stable/examples/bulk.html#bulk-insert


In [ ]:
# Bulk insert `embedded_docs` into the `collection` defined above -- should be a one-liner
<CODE_BLOCK_7>

print(f"Ingested {collection.count_documents({})} documents into the {COLLECTION_NAME} collection.")

# Step 5: Create a vector search index

In [ ]:
from utils import create_index, check_index_ready

In [ ]:
# Create vector index definition specifying:
# path: Path to the embeddings field
# numDimensions: Number of embedding dimensions- depends on the embedding model used
# similarity: Similarity metric. One of cosine, euclidean, dotProduct.
model = {
    "name": ATLAS_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 1024,
                "similarity": "cosine",
            }
        ]
    },
}

📚 Refer to the `utils.py` script under `notebooks/utils`

In [ ]:
# Use the `create_index` function from the `utils` module to create a vector search index with the above definition for the `collection` collection
create_index(collection, ATLAS_VECTOR_SEARCH_INDEX_NAME, model)

In [ ]:
# Use the `check_index_ready` function from the `utils` module to verify that the index was created and is in READY status before proceeding
check_index_ready(collection, ATLAS_VECTOR_SEARCH_INDEX_NAME)

In [ ]:
# Track progress of key steps-- DO NOT CHANGE
track_progress("vs_index_creation", "ai_rag_lab")

# Step 6: Perform vector search on your data


### Define a vector search function

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#ann-examples (Refer to the "Basic Example")


In [ ]:
# Define a function to retrieve relevant documents for a user query using vector search
def vector_search(user_query: str) -> List[Dict]:
    """
    Retrieve relevant documents for a user query using vector search.

    Args:
    user_query (str): The user's query string.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the `user_query` using the `get_embeddings` function defined in Step 3
    # NOTE: Wrap the user_query in a list since the function expects a list of strings
    # `input_type` should be set to "query" since we are embedding the query
    query_embedding = <CODE_BLOCK_8>

    # Define an aggregation pipeline consisting of a $vectorSearch stage, followed by a $project stage
    # Set the number of candidates to 150 and only return the top 5 documents from the vector search
    # In the $project stage, exclude the `_id` field and include these fields: `body`, `metadata.productName`, `metadata.contentType`, `updated` and `vectorSearchScore`
    # NOTE: Use variables defined previously for the `index`, `queryVector` and `path` fields in the $vectorSearch stage
    pipeline = <CODE_BLOCK_9>

    # Execute the aggregation `pipeline` and store the results in `results`
    results = <CODE_BLOCK_10>
    return list(results)

### Run vector search queries


In [ ]:
vector_search("What are some best practices for data backups in MongoDB?")

In [ ]:
vector_search("How to resolve alerts in MongoDB?")

# 🦹‍♀️ Combine pre-filtering with vector search

### Filter for documents where the product name is `MongoDB Atlas`

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#about-the-filter-type

In [ ]:
# Modify the vector search index `model` from Step 6 to include the `metadata.productName` field as a `filter` field
model = <CODE_BLOCK_11>

In [ ]:
# Use the `create_index` function from the `utils` module to re-create the vector search index with the modified model
create_index(collection, ATLAS_VECTOR_SEARCH_INDEX_NAME, model)

In [ ]:
# Use the `check_index_ready` function from the `utils` module to verify that the index has the right filter fields and is in READY status before proceeding
check_index_ready(collection, ATLAS_VECTOR_SEARCH_INDEX_NAME)

In [ ]:
# Embed the user query
query_embedding = get_embeddings(
    ["What are some best practices for data backups in MongoDB?"], "query"
)

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#ann-examples (Refer to the "Filter Example")

In [ ]:
# Modify the aggregation pipeline defined in Step 6 to:
# Include a filter in the $vectorSearch stage for documents where the `metadata.productName` field has the value "MongoDB Atlas".
pipeline = <CODE_BLOCK_12>

In [ ]:
# Execute the aggregation pipeline and view the results
results = collection.aggregate(pipeline)
list(results)

### Filter on documents which have been updated on or after `2024-05-19` and where the content type is `Tutorial`

In [ ]:
# Modify the vector search index `model` from Step 6 to include `metadata.contentType` and `updated` as `filter` fields
model = <CODE_BLOCK_13>

In [ ]:
# Use the `create_index` function from the `utils` module to re-create the vector search index with the modified model
create_index(collection, ATLAS_VECTOR_SEARCH_INDEX_NAME, model)

In [ ]:
# Use the `check_index_ready` function from the `utils` module to verify that the index has the right filter fields and is in READY status before proceeding
check_index_ready(collection, ATLAS_VECTOR_SEARCH_INDEX_NAME)

In [ ]:
# Embed the user query
query_embedding = get_embeddings(
    ["What are some best practices for data backups in MongoDB?"], "query"
)

In [ ]:
# Modify the aggregation pipeline defined in Step 6 to:
# Include a filter in the $vectorSearch stage for documents where the `metadata.contentType` field is "Tutorial" AND the `updated` field is greater than or equal to "2024-05-19".
# HINT: Use the $gte operator to check >= and the $and operator to combine the conditions.
pipeline = <CODE_BLOCK_14>

In [ ]:
# Execute the aggregation pipeline and view the results
results = collection.aggregate(pipeline)
list(results)

# Step 7: Build the RAG application


In [ ]:
import requests

### Define a function to create the chat prompt

In [ ]:
# Define a function to create the user prompt for our RAG application
def create_prompt(user_query: str) -> str:
    """
    Create a chat prompt that includes the user query and retrieved context.

    Args:
        user_query (str): The user's query string.

    Returns:
        str: The chat prompt string.
    """
    # Retrieve the most relevant documents for the `user_query` using the `vector_search` function defined in Step 6
    context = <CODE_BLOCK_15>
    # Join the retrieved documents into a single string, where each document is separated by two new lines ("\n\n")
    context = "\n\n".join([doc.get('body') for doc in context])
    # Prompt consisting of the question and relevant context to answer it
    prompt = f"Answer the question based only on the following context. If the context is empty, say I DON'T KNOW\n\nContext:\n{context}\n\nQuestion:{user_query}"
    return prompt

### Define a function to answer user queries

In [ ]:
# Define a function to answer user queries
def generate_answer(user_query: str) -> None:
    """
    Generate an answer to the user query.

    Args:
        user_query (str): The user's query string.
    """
    # Use the `create_prompt` function above to create a chat prompt
    prompt = <CODE_BLOCK_16>
    # Format the message to the LLM in the format [{"role": <role_value>, "content": <content_value>}]
    # The role value for user messages must be "user"
    # Use the `prompt` created above to populate the `content` field in the chat message
    messages = <CODE_BLOCK_17>
    # Send the chat messages to the AI model proxy get back an LLM response
    response = requests.post(url=PROXY_ENDPOINT, json={"task": "completion", "data": messages})
    if response.status_code == 200:
        # Print the final answer
        print(response.json()["text"])
    else:
        # Print the error message
        print(response.json()["error"])

### Query the RAG application


In [ ]:
generate_answer("What are some best practices for data backups in MongoDB?")

In [ ]:
# Notice that the LLM does not remember the conversation history at this stage
generate_answer("What did I just ask you?")

# 🦹‍♀️ Re-rank retrieved results


📚 https://docs.voyageai.com/docs/reranker#python-api (See Example)

In [ ]:
# Add a re-ranking step to the following function
def create_prompt(user_query: str) -> str:
    """
    Create a chat prompt that includes the user query and retrieved context.

    Args:
        user_query (str): The user's query string.

    Returns:
        str: The chat prompt string.
    """
    # Retrieve the most relevant documents for the `user_query` using the `vector_search` function defined in Step 6
    context = vector_search(user_query)
    # Extract the "body" field from each document in `context`
    documents = [d.get("body") for d in context]
    # Use the `rerank` method of the Voyage AI API to re-rank the `documents` witht he following arguments:
    # model: "rerank-2.5"
    # top_k: 5
    reranked_documents = <CODE_BLOCK_18>
    # Join the re-ranked documents into a single string, where each document is separated by two new lines ("\n\n")
    context = "\n\n".join([d.document for d in reranked_documents.results])
    # Prompt consisting of the question and relevant context to answer it
    prompt = f"Answer the question based only on the following context. If the context is empty, say I DON'T KNOW\n\nContext:\n{context}\n\nQuestion:{user_query}"
    return prompt

In [ ]:
# Note the impact of re-ranking on the generated answer
# You might not see a difference in this example since we are only re-ranking 5 documents
# In practice, you would send a larger number of documents to the re-ranker, and get the top few AFTER reranking
generate_answer("What are some best practices for data backups in MongoDB?")

# Step 8: Add memory to the RAG application


In [ ]:
from datetime import datetime

In [ ]:
history_collection = mongodb_client[DB_NAME]["chat_history"]

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.create_index


In [ ]:
# Create an index on the key `session_id` for the `history_collection` collection
<CODE_BLOCK_19>

### Define a function to store chat messages in MongoDB

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.insert_one

In [ ]:
def store_chat_message(session_id: str, role: str, content: str) -> None:
    """
    Store a chat message in a MongoDB collection.

    Args:
        session_id (str): Session ID of the message.
        role (str): Role for the message. One of `system`, `user` or `assistant`.
        content (str): Content of the message.
    """
    # Create a message object with `session_id`, `role`, `content` and `timestamp` fields
    # `timestamp` should be set the current timestamp
    message = {
        "session_id": session_id,
        "role": role,
        "content": content,
        "timestamp": datetime.now(),
    }
    # Insert the `message` into the `history_collection` collection
    <CODE_BLOCK_20>

### Define a function to retrieve chat history from MongoDB

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/cursor.html#pymongo.cursor.Cursor.sort

In [ ]:
def retrieve_session_history(session_id: str) -> List:
    """
    Retrieve chat message history for a particular session.

    Args:
        session_id (str): Session ID to retrieve chat message history for.

    Returns:
        List: List of chat messages.
    """
    # Query the `history_collection` collection for documents where the "session_id" field has the value of the input `session_id`
    # Sort the results in increasing order of the values in `timestamp` field
    cursor =  <CODE_BLOCK_21>

    if cursor:
        # Iterate through the cursor and extract the `role` and `content` field from each entry
        # Then format each entry as: {"role": <role_value>, "content": <content_value>}
        messages = [{"role": msg["role"], "content": msg["content"]} for msg in cursor]
    else:
        # If cursor is empty, return an empty list
        messages = []

    return messages

### Handle chat history in the `generate_answer` function

In [ ]:
def generate_answer(session_id: str, user_query: str) -> None:
    """
    Generate an answer to the user's query taking chat history into account.

    Args:
        session_id (str): Session ID to retrieve chat history for.
        user_query (str): The user's query string.
    """
    # Initialize list of messages to pass to the chat completion model
    messages = []

    # Retrieve documents relevant to the user query and convert them to a single string
    context = vector_search(user_query)
    context = "\n\n".join([d.get("body", "") for d in context])
    # Create a system prompt containing the retrieved context
    system_message = {
        "role": "user",
        "content": f"Answer the question based only on the following context. If the context is empty, say I DON'T KNOW\n\nContext:\n{context}",
    }
    # Append the system prompt to the `messages` list
    messages.append(system_message)

    # Use the `retrieve_session_history` function to retrieve message history from MongoDB for the session ID `session_id` 
    # And add all messages in the message history to the `messages` list 
    message_history = <CODE_BLOCK_22>
    messages.extend(message_history)

    # Format the user query in the format {"role": <role_value>, "content": <content_value>}
    # The role value for user messages must be "user"
    # And append the user message to the `messages` list
    user_message = <CODE_BLOCK_23>
    messages.append(user_message)

    # Send the chat messages to the AI model proxy to get back an LLM response
    response = requests.post(url=PROXY_ENDPOINT, json={"task": "completion", "data": messages})

    # Extract the answer from the response
    answer = response.json()["text"]

    # Use the `store_chat_message` function to store the user message and also the generated answer in the message history collection
    # The role value for user messages is "user", and "assistant" for the generated answer
    <CODE_BLOCK_24>

    print(answer)

In [ ]:
generate_answer(
    session_id="1",
    user_query="What are some best practices for data backups in MongoDB?",
)

In [ ]:
generate_answer(
    session_id="1",
    user_query="What did I just ask you?",
)